In [13]:
# 사전학습 모델 가져오기
from tensorflow.keras.applications import VGG16

base_model = VGG16(include_top=True, #특성추출기, 분류기 둘다 가져옴
                  weights='imagenet', #이미지넷 가중치 사용
                  input_shape=(224, 224, 3)) #입력 이미지 사이즈

In [15]:
# 훈련할 수 있는 가중치 갯수
len(base_model.trainable_weights) #32

32

In [16]:
# 전체 모델의 가중치 동결
base_model.trainable = False

In [17]:
# 동결한 후 가중치 갯수 확인
len(base_model.trainable_weights) #0

0

# CIFAR-10을 이용한 전이학습

특성추출기, 분류기를 가져와서 전체 동결 후 예측

In [18]:
# CIFAR-10 이미지 불러오기
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [20]:
# 베이스 모델 가져오기
from keras.applications.resnet50 import ResNet50
base_model = ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3))
base_model.summary()

102967424/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_4[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                    

In [21]:
# 컴파일
base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [42]:
# 예측, 필요한 경우 이미지 차원 및 사이즈 변경
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np

inputs = tf.expand_dims(x_test[0], axis=0) #차원추가 (None,32,32,3)
x = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)(inputs) #사이즈 변경
x = preprocess_input(x) #정규화
decode_predictions(base_model.predict(x), top=1)

1/1 [==============================] - 0s 436ms/step


[[('n01704323', 'triceratops', 0.10917217)]]

# 특성추출기만 가져와서 분류기를 붙인 후에 추출기 동결후 학습, 추출기 일부 해제후 학습

In [69]:
# CIFAR-10 이미지 불러오기
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [70]:
# ResNet50의 특성추출기만 가져오기
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_11[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [72]:
# 모델에 분류기 추가
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Lambda, Flatten, Dense
from tensorflow.keras.applications.resnet50 import preprocess_input

model = Sequential()
model.add(Input(shape=(32, 32, 3)))
model.add(Lambda(lambda image: tf.image.resize(image, (224, 224))))
model.add(Lambda(preprocess_input))
model.add(base_model)
model.add(Flatten())
model.add(Dense(10, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_4 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 lambda_5 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_5 (Flatten)         (None, 100352)            0         
                                                                 
 dense_5 (Dense)             (None, 10)                1003530   
                                                                 
Total params: 24591242 (93.81 MB)
Trainable params: 24538122 (93.61 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [73]:
# # 분류기를 추가하여 모델 생성
# import tensorflow as tf
# from keras import Input, Model
# from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
# inputs = Input(shape=(32,32,3))
# x = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)(inputs) #사이즈 변경
# x = tf.keras.applications.resnet50.preprocess_input(inputs) #정규화
# x = base_model(x, training = False)
# x = Flatten()(x)
# outputs = Dense(10, activation = 'softmax')(x)
# model = Model(inputs, outputs)
# model.summary()

In [74]:
# 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [75]:
# 체크포인트, 조기종료 설정
checkpoint = tf.keras.callbacks.ModelCheckpoint('cifar10_transfer_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [68]:
# 특성추출기 동결
base_model.trainable = False

# 모델 학습
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), callbacks=[checkpoint, earlystopping])

# 모델 저장
model.save('cifar10_transfer_model.h5')

Epoch 1/100
 283/1563 [====>.........................] - ETA: 2:23:44 - loss: 2.3511 - accuracy: 0.1116

KeyboardInterrupt: 

In [ ]:
# 히스토리 그래프
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
plt.legend()
plt.subplot(122)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')
plt.legend()
plt.show()

# Fine-Tuning

In [ ]:
# 모델 컴파일 (학습률 0.001)
from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(learning_rate= 0.001),
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
# 체크포인트, 조기 종료
save_file_name = 'cifar10_transfer_model.h5'
checkpoint = ModelCheckpoint(save_file_name, monitor='val_loss',
                             verbose=0, save_best_only=True, mode='auto')
earlystopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# 일부 레이어 해제
for layer in base_model.layers[143:]:	# 143층부터의 동결해제
  layer.trainable = True

In [ ]:
# 모델 학습
history_2 = model.fit(x_train, y_train, epochs = 100, validation_data=(x_test, y_test), batch_size= 256, callbacks=[checkpoint, earlystopping])

In [ ]:
# 모델 저장
model.save("cifar10_transfer_model.keras")

In [ ]:
# 히스토리 그래프
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.title('Cross Entropy Loss')
plt.plot(history_2.history['loss'], color='blue', label='train')
plt.plot(history_2.history['val_loss'], color='orange', label='test')
plt.legend()
plt.subplot(122)
plt.title('Classification Accuracy')
plt.plot(history_2.history['accuracy'], color='blue', label='train')
plt.plot(history_2.history['val_accuracy'], color='orange', label='test')
plt.legend()
plt.show()

# 특성추출기를 가져와서 분류기를 추가한 후 이미지 데이터를 증강하여 학습

In [1]:
# CIFAR-10 이미지 불러오기
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [2]:
# ResNet50의 특성추출기만 가져오기
from keras.applications.resnet50 import ResNet50
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [3]:
# 분류기 추가하여 모델 생성
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Lambda, Flatten, Dense
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

model_2 = Sequential()
model_2.add(Input(shape=(32, 32, 3)))
model_2.add(Lambda(lambda image: tf.image.resize(image, (224, 224))))
model_2.add(Lambda(preprocess_input))
model_2.add(base_model)
model_2.add(Flatten())
model_2.add(Dense(10, activation='softmax'))
model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 lambda_1 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 10)                1003530   
                                                                 
Total params: 24591242 (93.81 MB)
Trainable params: 24538122 (93.61 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [4]:
# 컴파일
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# 체크포인트, 조기종료 설정
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint =ModelCheckpoint('cifar10_transfer_model.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
earlystopping = EarlyStopping(monitor='val_loss', patience=20)

In [6]:
# 이미지 데이터 증강 설정
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)
train_flow = datagen.flow(x_train, y_train, batch_size=32,subset='training')
test_flow = datagen.flow(x_train, y_train, batch_size=8, subset='validation')

In [7]:
# 베이스모델 동결
base_model.trainable = False

# 모델 학습
epochs=100
history = model_2.fit(train_flow,
                      validation_data=test_flow,
                      steps_per_epoch=len(x_train) / 32, #한epoch내 배치수
                      epochs=epochs,
                      callbacks=[checkpoint, earlystopping])

# 모델 저장
# 모델 저장
model.save('cifar10_fine_tuning_2.h5')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1862: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1872: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# 예측, 필요한 경우 이미지 차원 및 사이즈 변경
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np

inputs = tf.expand_dims(x_test[0], axis=0) #차원추가 (None,32,32,3)
x = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)(inputs) #사이즈 변경
x = preprocess_input(x) #정규화
decode_predictions(model_2.predict(x), top=1)

In [44]:
# 코랩환경 옮기고 싶을 때
!pip freeze > requirements.txt #코랩환경을 텍스트 파일로 옮기고
!pip install -r requirements.txt #로컬에서 텍스트파일대로 설치